In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import datetime

import locale
locale.setlocale(locale.LC_ALL, 'ru_RU.UTF-8')

In [ ]:
sale_prices = pd.read_excel('Анализ цен Федору.xlsx', sheet_name='рентаб-ть цена продажи')
sale_prices.head()

In [ ]:
categories = sale_prices.iloc[5:16,0].to_frame('category')
categories

In [ ]:
sale_prices = sale_prices.iloc[1:, 1:]
sale_prices.head()

In [ ]:
def str_to_period(input: str) -> pd.Period:
    months = {'Май': 'мая'}
    try:
        input_lst = input.split(sep=' ')[:-1]
        input_lst[0] = input_lst[0][:3]
        input_lst[0] = months.get(input_lst[0], input_lst[0])
        input = ' '.join(input_lst)
        dt = datetime.datetime.strptime(input, '%b %Y')
        return pd.Timestamp(dt).to_period('M')
    except Exception as e:
        print(e)
        breakpoint()

In [ ]:
t = sale_prices.copy()
columns = ['count', 'income', 'payability', 'gross_margin', 'total']
results = []
while (t.shape[1] > 0):
    dt = str_to_period(t.iloc[0, 0])
    batch = t.iloc[2:,:5]
    batch.columns = columns
    batch['report_dt'] = dt
    batch = categories.join(batch)
    result = batch.dropna()
    results += result.to_dict('r')
    t.drop(columns=t.columns[:5], inplace=True)    

In [ ]:
sale_prices = pd.DataFrame(results)
sale_prices

In [ ]:
def series_to_df_offer_id_and_date(nomenclature: pd.Series) -> pd.DataFrame:
    tmp = nomenclature.str.split(expand=True)
    tmp.iloc[:, -2] = pd.to_datetime(tmp.iloc[:, -2] + ' ' + tmp.iloc[:, -1])
    tmp = tmp.iloc[:, [-4, -2]]
    tmp.columns = ['offer_id', 'report_dt']
    return tmp

In [ ]:
def insert_and_remove_columns_dt(dt, tmp: pd.DataFrame) -> pd.DataFrame:
    dt_copy = dt.copy()
    dt_copy['offer_id'] = tmp.offer_id
    dt_copy['report_dt'] = tmp.report_dt
    dt_copy = dt_copy.drop(columns=['nomenclature'])
    return dt_copy

In [ ]:
def edit_lifting_dt(dt: pd.DataFrame) -> pd.DataFrame:
    dt_copy = dt.iloc[8:, [0, 1, 3]]
    dt_copy.columns = ['category', 'nomenclature', 'count']
    tmp = series_to_df_offer_id_and_date(dt_copy.nomenclature)
    dt_copy = insert_and_remove_columns_dt(dt_copy, tmp)
    return dt_copy

In [ ]:
lifting_price = pd.read_excel('Анализ цен Федору.xlsx', sheet_name='отмена (ЦЕНА)')
lifting_price.head(15)

In [ ]:
lifting_price = edit_lifting_dt(lifting_price)
lifting_price.head()

In [ ]:
lifting_deficit = pd.read_excel('Анализ цен Федору.xlsx', sheet_name='отмена (ДЕФИЦИТ)')
lifting_deficit.head(15)

In [ ]:
lifting_deficit = edit_lifting_dt(lifting_deficit)
lifting_deficit.head()

In [ ]:
product_movement = pd.read_excel('Анализ цен Федору.xlsx', sheet_name='Движение товара')
product_movement.head()

In [ ]:
product_movement.columns = ['category',
                'nomenclature',
                'opening_balance',
                'inflow',
                'consumption',
                'final_balance']
product_movement = product_movement.dropna(subset=['category'])
product_movement.head()

In [ ]:
full_warehouse = pd.read_excel('Развернутые данные по гайке DIN934.xlsx', sheet_name='движение товара')

In [ ]:
full_warehouse.head(10)

In [ ]:
head_table = full_warehouse.iloc[[6,7],:]
full_warehouse.drop(columns=full_warehouse.columns[[1, 2, 3, 5]], index=range(10), inplace=True)
full_warehouse.drop(index=full_warehouse.index[-1], inplace=True)

In [ ]:
categories = product_movement.category.unique()
categories

In [ ]:
full_warehouse.index -= 10

In [ ]:
start_index = 0
full_warehouse['category'] = np.nan
category = ''
drop_indexes = []
for i in range(full_warehouse.shape[0]):
    row = full_warehouse.iloc[i,:]
    if row[0] in categories:
        full_warehouse.category[start_index: i] = category
        drop_indexes.append(start_index)
        category = row[0]
        start_index = i

drop_indexes.append(start_index)
full_warehouse.category[start_index:] = category

full_warehouse.drop(index=drop_indexes, inplace=True)

In [ ]:
t = full_warehouse[full_warehouse.columns[0]]
result = []
for s in t:
    row = []
    s = s.split(sep=' ')
    result.append([' '.join(s[:-4]), s[-4], pd.to_datetime(s[-2] + ' ' + s[-1])])
    
split_nomeclature = pd.DataFrame(result)

In [ ]:
full_warehouse.index = range(full_warehouse.shape[0])
full_warehouse.drop(columns=full_warehouse.columns[0], inplace=True)
full_warehouse = full_warehouse.join(split_nomeclature)
full_warehouse.columns = ['opening_balance', 'income', 'consumption', 'final_balance', 'mean_price', 'prime_cost', 'category', 'nomenclature', 'doc_id', 'report_dttm']
full_warehouse.head()

In [ ]:
t = {'sales': sale_prices, 
     'warehouse': product_movement, 
     'cancels': lifting_price,
     'deficit': lifting_deficit,
     'full_warehouse': full_warehouse}

In [ ]:
import pickle

In [ ]:
with open('df_dump.dat', 'bw') as file:
    pickle.dump(t, file, protocol=pickle.HIGHEST_PROTOCOL)